In [19]:
import torch
from torch import nn
from src.utils.preprocess_utils import notes_to_midi, midi_to_note_indices, one_hot_encode
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [20]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, input_size]
        out, _ = self.lstm(x)  # out: [batch_size, sequence_length, hidden_size]
        out = self.fc(out)
        return out

In [21]:
# Load model
model_name = "2025-01-04_22-12-29_h128_l2_sl60"
model = torch.load(f'weights/model_1/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_30496\1303454779.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_1/{model_name}')


LSTMModel(
  (lstm): LSTM(85, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [22]:
# Load input
input = midi_to_note_indices('data/input.mid', 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [23]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_sequence_one_hot = one_hot_encode(start_sequence, num_notes=num_notes)
    start_tensor = torch.tensor(start_sequence_one_hot, dtype=torch.float32).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(one_hot_encode(generated_sequence_tensor, num_notes=num_notes), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 40, 70, 11, 72, 73, 81, 74, 75, 78, 77, 76, 64, 76, 68, 7, 71, 74, 59, 80, 71, 80, 72, 74, 78, 76, 79, 80, 79, 72, 77, 74, 76, 78, 75, 80, 77, 74, 79, 76, 73, 78, 77, 80, 74, 76, 70, 72, 68, 66, 61, 73, 80, 74, 62, 61, 49, 53, 38, 50, 62, 61, 49, 55, 3, 50, 53, 49, 52, 54, 53, 50, 48, 47, 28, 7, 50, 38, 49, 40, 49, 50, 38, 53, 41, 55, 53, 52, 54, 15, 3, 55, 50, 54, 56, 52, 54, 50, 48, 47, 49, 44, 40, 38, 31, 35, 19, 7, 19, 7, 34, 38, 23, 11, 35, 38, 30, 18, 30, 3, 15, 3, 15, 3, 31, 3, 15, 3, 42, 30, 7, 8, 20, 8, 41, 20, 8, 35, 32, 1, 13, 1, 13, 41, 4, 16, 4, 16, 4, 37, 8, 41, 25, 13, 48, 41, 29, 33, 48, 54, 13, 57, 49, 53, 52, 64, 61, 49, 61, 73, 61, 49, 62, 61, 73, 61, 73, 77, 78, 68, 71, 73, 72, 75, 64, 77, 76, 75, 76, 78, 19, 7, 71, 74, 68, 56, 78, 68, 79, 77, 75]


In [24]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_1/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_1/2025-01-04_22-23-17_t0.3_w200_M_2025-01-04_22-12-29_h128_l2_sl60.mid
